In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy
from scipy.stats import norm

In [18]:
#read data from "MacroSeries.txt"

macro_data = pd.read_csv("MacroSeries.txt", header = None)
macro_data.columns = ["c", "k", "w", "r", "y"]
c_vec = np.array(macro_data["c"])
k_vec = np.array(macro_data["k"])
w_vec = np.array(macro_data["w"])
r_vec = np.array(macro_data["r"])
y_vec = np.array(macro_data["y"])


The model is given by 
$$c_t^{-1} - \beta \mathbb{E}[r_{t+1} (c_{t+1})^{-1}] = 0$$
$$c_t + k_{t+1} - w_t - r_t k_t = 0$$
$$w_t - (1-\alpha)e^{z_t} (k_t)^\alpha = 0$$
$$r_t - \alpha e^{z_t} (k_t) ^{\alpha - 1} = 0$$
$$z_t = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t $$
$$y_t = e ^{z_t} (k_t)^\alpha $$



In [12]:
#assume beta = 0.99

Observe that from (3), $$(1-\alpha)e^{z_t}(k_t)^\alpha=w_t$$
$$e^{z_t} = \frac{w_t}{(1-\alpha)(k_t)^\alpha}$$
$$z_t = \ln\{\frac{w_t}{(1-\alpha)(k_t)^\alpha}\} = \ln w_t - \ln (1-\alpha) - \alpha \ln k_t$$
$$z_t \in N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2).$$ Use mle to estimate $\alpha, \rho, \mu, \sigma$

In [27]:
#we need some initial guess
alpha, rho, mu, sigma = (0.5, 0.5, 2, 1.2)
z_series = np.zeros(100)
z_series[0] = mu

def get_z(w_vec, k_vec, alpha):
    return np.log(w_vec) - np.log(1-alpha) - alpha*np.log(k_vec)

def logpdf_of_z(z, rho, mu, sigma):
    z[0] = mu
    mean = rho * z[:-1] + (1 - rho) * mu
    logpdf = norm.logpdf(z, np.hstack([mu, mean]), sigma)
    return logpdf

def crit1(param_vec, args):
    alpha, rho, mu, sigma = param_vec
    w_vec, c_vec = args
    z_vec = get_z(w_vec, k_vec, alpha)
    return - logpdf_of_z(z_vec, rho, mu, sigma).sum()

param_vec_guess = [0.5, 0.5, 1.2, 1.2]
args1 = [w_vec, c_vec]
soln1 = opt.minimize(crit1, param_vec_guess, args = args1, method = "L-BFGS-B", bounds = ((1e-6, 1-1e-6), (-1+1e-6, 1-1e-6),(1e-6, None), (1e-6, None)))
soln1

      fun: -95.55298518128292
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00996749,  0.00024869,  0.00104734, -0.00069917])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 345
      nit: 44
   status: 0
  success: True
        x: array([0.85065477, 0.51750877, 4.56654189, 0.09306392])

In [31]:
alpha_mle1, rho_mle1, mu_mle1, sigma_mle1 = soln1.x
soln1.hess_inv.todense()

array([[ 7.71329813e+00, -2.51970208e+00, -7.03536630e+01,
         4.30875782e-01],
       [-2.51970208e+00,  4.05889118e+00,  2.23651029e+01,
        -5.64595257e-01],
       [-7.03536630e+01,  2.23651029e+01,  6.41820296e+02,
        -3.84897833e+00],
       [ 4.30875782e-01, -5.64595257e-01, -3.84897833e+00,
         7.99307392e-02]])

Use another equation to get a vector of z.
$$r_t - \alpha e^{z_t} (k_t) ^{\alpha - 1} = 0$$
$$r_t = \alpha e^{z_t} (k_t) ^{\alpha - 1} $$
$$ e^{z_t} = \frac{r_t}{\alpha (k_t) ^{\alpha - 1}}$$
$$z_t = \ln r_t - \ln\alpha - (\alpha-1)\ln k_t $$

In [33]:
def get_z2(r_vec, k_vec, alpha):
    return np.log(r_vec) - np.log(alpha) - (alpha -1)* np.log(k_vec)

#the log pdf of z2 is the same

def crit2(param_vec, args):
    alpha, rho, mu, sigma = param_vec
    r_vec, k_vec = args
    z_vec = get_z2(r_vec, k_vec, alpha)
    return - logpdf_of_z(z_vec, rho, mu, sigma).sum()

#init guess is the same
args2 = [r_vec, k_vec]
soln2 = opt.minimize(crit2, param_vec_guess, args = args2, method = "L-BFGS-B", bounds = ((1e-6, 1-1e-6), (-1+1e-6, 1-1e-6),(1e-6, None), (1e-6, None)))
soln2

      fun: -95.55298514141421
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.02065832, -0.00051159,  0.00124487, -0.01112852])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 255
      nit: 34
   status: 0
  success: True
        x: array([0.85063667, 0.51752431, 2.50432769, 0.09306346])

In [34]:
alpha_mle2, rho_mle2, mu_mle2, sigma_mle2 = soln2.x
soln2.hess_inv.todense()

array([[ 6.02021347e+01, -1.80736232e+02, -1.02025110e+03,
         4.99806543e+00],
       [-1.80736232e+02,  5.47312999e+02,  3.06275142e+03,
        -1.51468642e+01],
       [-1.02025110e+03,  3.06275142e+03,  1.72902983e+04,
        -8.46966209e+01],
       [ 4.99806543e+00, -1.51468642e+01, -8.46966209e+01,
         4.19269585e-01]])

In [38]:
# get probability
z_star = get_z2(1, 7500000, alpha_mle1)
mean = rho_mle1 * 10 + (1 - rho_mle1) * mu_mle1
print('Probability : ', 1-norm.cdf(z_star, mean, sigma_mle1))

Probability :  1.0
